In [1]:
# import libraries

import pandas as pd


import numpy as np

import warnings
warnings.filterwarnings ('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import seaborn as sns

import matplotlib.pyplot as plt

from geopy.geocoders import Nominatim

In [2]:
data_prior2_ori = pd.read_csv('../Proyecto_Arraigo_Recomender_System/data/Urbanitas_prior2.csv')


data_prior2 = data_prior2_ori.copy()

In [3]:
data_prior2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10618 entries, 0 to 10617
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Unnamed: 0                10618 non-null  int64 
 1   id                        10617 non-null  object
 2   fecha                     10618 non-null  object
 3   pais_de_residencia        10590 non-null  object
 4   provincia_de_espana       10590 non-null  object
 5   ciudad_residencia         10590 non-null  object
 6   situacion_en_espana       10571 non-null  object
 7   profesion                 10515 non-null  object
 8   autonomia_economica       8407 non-null   object
 9   familia_edades            9025 non-null   object
 10  coche_propio              10544 non-null  object
 11  carnet_coche              8924 non-null   object
 12  mascota                   10618 non-null  object
 13  presupuesto_alquiler      10577 non-null  object
 14  primera_segunda_vivien

In [4]:
porcentaje_nulos = data_prior2.isnull().sum() / data_prior2.count() * 100
porcentaje_nulos.sort_values(ascending=False)

autonomia_economica         26.299512
carnet_coche                18.982519
familia_edades              17.650970
profesion                    0.979553
predisposicion_pueblo        0.787850
coche_propio                 0.701821
primera_segunda_vivienda     0.587344
situacion_en_espana          0.444613
presupuesto_alquiler         0.387634
ciudad_residencia            0.264400
provincia_de_espana          0.264400
pais_de_residencia           0.264400
id                           0.009419
fecha                        0.000000
mascota                      0.000000
zona_interes                 0.000000
Unnamed: 0                   0.000000
dtype: float64

LIMPIEZA AUTONOMIA_ECONOMICA

In [5]:
data_prior2.autonomia_economica.unique()

array([nan, '3 meses', '6 meses', '1 año', 'Más de 1 año', '6 ???????',
       'Hasta 12 meses', 'Hasta 3 meses', 'Hasta 6 meses',
       'Más de 12 meses',
       'No dispongo de autonomía económica en estos momentos',
       'Dispongo de capacidad económica regular (jubilado, trabajo a distancia, trabajo fijo...)',
       'Más de 6 meses', '1 ano', '???? ?? ???', '3 ??????', '3 mois'],
      dtype=object)

In [6]:
data_prior2.autonomia_economica.isnull().sum()
#desconocemos la autonomía económica de 2211 personas -> preguntar al cliente

2211

In [7]:
data_prior2 = data_prior2.replace({'autonomia_economica': {  '1 ano': '1 año',
                                                             '6 ???????': '6 meses',
                                                             'Hasta 12 meses' : '1 año',
                                                             'Hasta 3 meses' :'3 meses',
                                                             'Hasta 6 meses' :'6 meses',
                                                             'Más de 12 meses' :'Más de 1 año',
                                                             'No dispongo de autonomía económica en estos momentos' :'3 meses',
                                                             'Dispongo de capacidad económica regular (jubilado, trabajo a distancia, trabajo fijo...)' :'3 meses',
                                                             'Más de 6 meses' :'1 año',
                                                             '3 ??????' :'3 meses',
                                                             '3 mois': '3 meses',

                                                      }})

data_prior2.autonomia_economica.unique()

array([nan, '3 meses', '6 meses', '1 año', 'Más de 1 año', '???? ?? ???'],
      dtype=object)

LIMPIEZA CARNET_COCHE

In [8]:
data_prior2.carnet_coche.unique()

array(['Si', 'No', '???', nan, 'sim', 'Non', '??', 'Não', 'Sim', 'TAk'],
      dtype=object)

In [9]:
#se entiende que si en el momento en que se contestó la pregunta era abierta y no fue respondida como un 'no'

data_prior2.fillna({'carnet_coche':'No'}, inplace=True)


In [10]:
data_prior2 = data_prior2.replace({'carnet_coche': {'sim': 'Si',
                                                    'Não': 'No',
                                                    'Non' : 'No',
                                                    'TAk' :'Si',
                                                    'Tak' :'Si',
                                                     'Sim' :'Si',
                                                     '???' :'Si',
                                                     '??' :'No',
                                                      }})

#los replacements de las ??? (cirílico) se han deducido por los datos del csv

In [11]:
data_prior2.carnet_coche.unique()

array(['Si', 'No'], dtype=object)

LIMPIEZA COCHE_PROPIO

In [12]:
data_prior2.coche_propio.unique()

array(['No', 'Si', '???', nan, 'Sí', 'sí', 'sim', 'Non', '??', 'Sim',
       'TAk', 'Não', 'NIE'], dtype=object)

In [13]:
#se entiende que si en el momento en que se contestó la pregunta era abierta y no fue respondida como un 'no'

data_prior2.fillna({'coche_propio':'No'}, inplace=True)

In [14]:
data_prior2 = data_prior2.replace({'coche_propio': {'sim': 'Si',
                                                    'Não': 'No',
                                                    'Non' : 'No',
                                                    'TAk' :'Si',
                                                     'Sim' :'Si',
                                                     '???' :'Si',
                                                     '??' :'No',
                                                     'NIE' :'No',
                                                     'Sí' :'Si',
                                                     'sí' :'Si',
                                                      }})

In [15]:
data_prior2.coche_propio.unique()

array(['No', 'Si'], dtype=object)

LIMPIEZA PREDISPOSICION_PUEBLO

In [16]:
data_prior2.predisposicion_pueblo.unique()

array([nan, '3 meses', 'Aún me lo estoy pensando', '6 meses', '1 mes',
       '1 ?????', 'Más de 6 meses', 'En los próximos 6 meses',
       'Cuanto antes (hasta 3 meses)', 'En los proximos 6 meses',
       'Cuanto antes  (hasta 3 meses)', 'Hasta 6 meses',
       'En el próximo mes', 'Cuanto antes',
       'Cuando termine nuestro contrato con el actual inmueble. A partir de tres años.',
       'Necesito pensarlo más', '1 mês', '??? ????', '1 mois',
       'Ciagle o tym mysle'], dtype=object)

In [17]:
data_prior2.predisposicion_pueblo.isnull().sum()
#desconocemos la predisposición a mudarse a un pueblo de 83 personas -> como el cuestionario estaba abierto en su momento, damos por hecho que no lo tenían claro.

83

In [18]:
data_prior2 = data_prior2.replace({'predisposicion_pueblo': {'1 ?????': '1 mes',
                                                             'Más de 6 meses': '6 meses',
                                                             'En los próximos 6 meses' : 'Aún me lo estoy pensando',
                                                             'Cuanto antes (hasta 3 meses)' :'3 meses',
                                                             'Cuanto antes  (hasta 3 meses)' :'3 meses',
                                                             'Hasta 6 meses' :'6 meses',
                                                             'En el próximo mes' :'Aún me lo estoy pensando',
                                                             'Cuanto antes' :'Aún me lo estoy pensando',
                                                             'Cuando termine nuestro contrato con el actual inmueble. A partir de tres años.' :'Aún me lo estoy pensando',
                                                             'ecesito pensarlo más' :'Aún me lo estoy pensando',
                                                             '1 mês': '1 mes',
                                                             '??? ????' :'Aún me lo estoy pensando',
                                                             'Ciagle o tym mysle' :'Aún me lo estoy pensando',
                                                             '1 mois': '1 mes',
                                                             'Necesito pensarlo más' :'Aún me lo estoy pensando',
                                                             'En los proximos 6 meses' : 'Aún me lo estoy pensando',
                                                      }})

data_prior2.predisposicion_pueblo.unique()

array([nan, '3 meses', 'Aún me lo estoy pensando', '6 meses', '1 mes'],
      dtype=object)

In [19]:
data_prior2.fillna({'predisposicion_pueblo':'Aún me lo estoy pensando'}, inplace=True)

LIMPIEZA ID

In [20]:
data_prior2.fillna({'id':'U-000002'}, inplace=True) #tenemos identificado el id que faltaba

In [21]:
porcentaje_nulos = data_prior2.isnull().sum() / data_prior2.count() * 100
porcentaje_nulos.sort_values(ascending=False) 

autonomia_economica         26.299512
familia_edades              17.650970
profesion                    0.979553
primera_segunda_vivienda     0.587344
situacion_en_espana          0.444613
presupuesto_alquiler         0.387634
pais_de_residencia           0.264400
provincia_de_espana          0.264400
ciudad_residencia            0.264400
mascota                      0.000000
zona_interes                 0.000000
Unnamed: 0                   0.000000
carnet_coche                 0.000000
coche_propio                 0.000000
id                           0.000000
fecha                        0.000000
predisposicion_pueblo        0.000000
dtype: float64

LIMPIEZA FAMILIA_EDADES

In [22]:
data_prior2.familia_edades.unique()

array(['30 - 24 - 2', '37,37', '57, 56', ..., '37-28-13-7-4',
       '35, 33, 10, 8, 5, 3', '41, 32 y 6 meses'], dtype=object)

In [23]:
import re

data_prior2['familia_edades'] = data_prior2['familia_edades'].apply(lambda x: ','.join(re.findall(r'\d+', str(x))))


In [24]:
data_prior2.familia_edades.unique()

array(['30,24,2', '37,37', '57,56', ..., '37,28,13,7,4', '35,33,10,8,5,3',
       '41,32,6'], dtype=object)

In [25]:
familias_1 = data_prior2[data_prior2['familia_edades'].str.len() == 1]['familia_edades'].tolist()

In [26]:
familias_1[:10], data_prior2.familia_edades.isnull().sum() #hay gente que ha contestado que tiene cero años

(['0', '4', '0', '0', '0', '0', '0', '6', '3', '9'], 0)

In [27]:
data_prior2.familia_edades.value_counts()[:10]

      1654
0      235
30      83
25      65
31      61
32      59
27      59
28      58
45      56
34      55
Name: familia_edades, dtype: int64

In [28]:
data_prior2['numero_familiares'] = data_prior2['familia_edades'].apply(lambda x: len(x.split(',')))


In [29]:
data_prior2.numero_familiares.value_counts()

1     3907
2     2417
3     1798
4     1434
5      655
6      233
7       96
8       42
9       14
10       9
12       5
16       3
15       2
19       1
13       1
11       1
Name: numero_familiares, dtype: int64

LIMPIEZA PRIMERA_SEGUNDA_VIVIENDA 

In [30]:
data_prior2.primera_segunda_vivienda.unique()

array([nan, 'Primera residencia', 'Segunda residencia',
       '?????? ??????????', 'Primeira residência', '??????? ??????',
       'première résidence', 'primeira residência', 'pierwsza rezydencja',
       'pierwsze miejsce zamieszkania'], dtype=object)

In [31]:
data_prior2.primera_segunda_vivienda.isnull().sum()

#62 personas no han marcado si la casa de destino es para primera o segunda vivienda.

62

In [32]:
data_prior2 = data_prior2.replace({'primera_segunda_vivienda': {'Primeira residência': 'Primera residencia',
                                                             'Primeira residência': 'Primera residencia',
                                                             'première résidence' : 'Primera residencia',
                                                             'primeira residência' :'Primera residencia',
                                                             'pierwsza rezydencja' :'Primera residencia',
                                                             'pierwsze miejsce zamieszkania' :'Primera residencia',
                                                             '?????? ??????????' :'Primera residencia', #esto lo sé porque es ucraniano
                                                             '??????? ??????' :'Primera residencia',

                                                      }})

data_prior2.primera_segunda_vivienda.unique()

array([nan, 'Primera residencia', 'Segunda residencia'], dtype=object)

LIMPIEZA SITUACION_EN_ESPAÑA

In [33]:
data_prior2.situacion_en_espana.unique()

array(['Tengo nacionalidad española / de un país de la UE / permiso de residencia',
       'No estoy en ninguna de esas situaciones',
       '? ?? ???????? ?? ? ????? ?? ???? ????????',
       'Soy español o tengo la nacionalidad española/europea',
       'No estoy en ninguna de estas situaciones',
       'Tengo permiso de residencia en España',
       'Situación de protección de asilo', nan,
       'Soy español o tengo nacionalidad española/ europea',
       'Situación de protección internacional o asilo',
       'Autorización de permanencia en España por estudios',
       'Otro tipo de visado (Visado de Residencia para Emprendedores o Actividad Empresarial (REM), Visado de Residencia para Inversores de Capital (RIC), Visado de Residencia por Adquisición de Bienes Inmuebles (RIV), Visado de Residencia para Profesionales Altamente Cualificados (TAC), Visado de Residencia para Formación o Investigación (RIN), Visado de Residencia por Traslado Empresarial (TTI))',
       'Sin respuesta',

In [34]:
data_prior2 = data_prior2.replace({'situacion_en_espana': {'? ?? ???????? ?? ? ????? ?? ???? ????????': 'No estoy en ninguna de esas situaciones',
                                                             'Soy español o tengo la nacionalidad española/europea': 'Tengo nacionalidad española / de un país de la UE / permiso de residencia',
                                                             'No estoy en ninguna de estas situaciones' : 'No estoy en ninguna de esas situaciones',
                                                             'Tengo permiso de residencia en España' :'Tengo nacionalidad española / de un país de la UE / permiso de residencia',
                                                             'Situación de protección de asilo' :'Tengo nacionalidad española / de un país de la UE / permiso de residencia',
                                                             'Soy español o tengo nacionalidad española/ europea' :'Tengo nacionalidad española / de un país de la UE / permiso de residencia',
                                                             'Situación de protección internacional o asilo' :'Tengo nacionalidad española / de un país de la UE / permiso de residencia', #esto lo sé porque es ucraniano
                                                             'Autorización de permanencia en España por estudios' :'Tengo nacionalidad española / de un país de la UE / permiso de residencia',
                                                             'Otro tipo de visado (Visado de Residencia para Emprendedores o Actividad Empresarial (REM), Visado de Residencia para Inversores de Capital (RIC), Visado de Residencia por Adquisición de Bienes Inmuebles (RIV), Visado de Residencia para Profesionales Altamente Cualificados (TAC), Visado de Residencia para Formación o Investigación (RIN), Visado de Residencia por Traslado Empresarial (TTI))' :'Tengo nacionalidad española / de un país de la UE / permiso de residencia',
                                                             'Permiso de residencia/ trabajo (Asilo)' :'Tengo nacionalidad española / de un país de la UE / permiso de residencia',
                                                             'En tramite (de aqui a 6 meses)' :'No estoy en ninguna de esas situaciones',
                                                             'En tramite (de aqui a 6 meses)' :'No estoy en ninguna de esas situaciones',
                                                             'Otro tipo de visado (Para emprendedores o Actividad empresarial)':'Tengo nacionalidad española / de un país de la UE / permiso de residencia',
                                                             'Eu não estou em nenhuma dessas situações':'No estoy en ninguna de esas situaciones',
                                                             'Tenho nacionalidade espanhola / de um país da UE / autorização de residência':'Tengo nacionalidad española / de un país de la UE / permiso de residencia',
                                                             '??? ??? ?? ?? ?? ??? ???????': 'No estoy en ninguna de esas situaciones',
                                                             "J'ai la nationalité espagnole / d'un pays de l'UE / un permis de séjour":'Tengo nacionalidad española / de un país de la UE / permiso de residencia',
                                                             '? ???? ???? ????????? ??????????? / ?? ?????? ?? / ??? ?? ??????????':'Tengo nacionalidad española / de un país de la UE / permiso de residencia',
                                                             '? ?? ? ????? ?? ???? ????????':'No estoy en ninguna de esas situaciones',
                                                             'Mam obywatelstwo hiszpanskie / z kraju UE / pozwolenie na pobyt': 'Tengo nacionalidad española / de un país de la UE / permiso de residencia',
                                                             'Mam obywatelstwo hiszpanskie / z kraju UE / zezwolenie na pobyt': 'Tengo nacionalidad española / de un país de la UE / permiso de residencia',
                                                              'Sí':'Tengo nacionalidad española / de un país de la UE / permiso de residencia',
                                                             
                                                             
                                                             
                                                             
                                                             
                                                             
                                                             
                                                             

                                                      }})

data_prior2.situacion_en_espana.unique()

array(['Tengo nacionalidad española / de un país de la UE / permiso de residencia',
       'No estoy en ninguna de esas situaciones', nan, 'Sin respuesta'],
      dtype=object)

In [35]:
situacion_check = data_prior2.query("situacion_en_espana == 'Sin respuesta'")['pais_de_residencia'].value_counts()

In [42]:
#Como son países de la UE o España, reemplazamos por 'Tengo nacionalidad española / de un país de la UE / permiso de residencia'

data_prior2.loc[(data_prior2['situacion_en_espana'] == 'Sin respuesta') & (data_prior2['pais_de_residencia'].str.contains('España', case=False, regex=False)), 'situacion_en_espana'] = 'Tengo nacionalidad española / de un país de la UE / permiso de residencia'
data_prior2.loc[(data_prior2['situacion_en_espana'] == 'Sin respuesta') & (data_prior2['pais_de_residencia'].str.contains('Italia', case=False, regex=False)), 'situacion_en_espana'] = 'Tengo nacionalidad española / de un país de la UE / permiso de residencia'
data_prior2.loc[(data_prior2['situacion_en_espana'] == 'Sin respuesta') & (data_prior2['pais_de_residencia'].str.contains('France', case=False, regex=False)), 'situacion_en_espana'] = 'Tengo nacionalidad española / de un país de la UE / permiso de residencia'
data_prior2.loc[(data_prior2['situacion_en_espana'] == 'Sin respuesta') & (data_prior2['pais_de_residencia'].str.contains('Deutschland', case=False, regex=False)), 'situacion_en_espana'] = 'Tengo nacionalidad española / de un país de la UE / permiso de residencia'


In [46]:
#En este caso, como no son países que pertenezcan a la UE y no han respondido, damos por hecho que se encuentran en otra situación

data_prior2.loc[(data_prior2['situacion_en_espana'] == 'Sin respuesta') & (data_prior2['pais_de_residencia'].str.contains('Argentina', case=False, regex=False)), 'situacion_en_espana'] = 'No estoy en ninguna de esas situaciones'
data_prior2.loc[(data_prior2['situacion_en_espana'] == 'Sin respuesta') & (data_prior2['pais_de_residencia'].str.contains('Honduras', case=False, regex=False)), 'situacion_en_espana'] = 'No estoy en ninguna de esas situaciones'
data_prior2.loc[(data_prior2['situacion_en_espana'] == 'Sin respuesta') & (data_prior2['pais_de_residencia'].str.contains('México', case=False, regex=False)), 'situacion_en_espana'] = 'No estoy en ninguna de esas situaciones'
data_prior2.loc[(data_prior2['situacion_en_espana'] == 'Sin respuesta') & (data_prior2['pais_de_residencia'].str.contains('Colombia', case=False, regex=False)), 'situacion_en_espana'] = 'No estoy en ninguna de esas situaciones'



In [47]:
situacion_check = data_prior2.query("situacion_en_espana == 'Sin respuesta'")['pais_de_residencia'].value_counts()


situacion_check

Series([], Name: pais_de_residencia, dtype: int64)